In [107]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.


from subprocess import check_output

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
# from sklearn.LDA import LDA
from sklearn.decomposition import LatentDirichletAllocation

import gensim
from gensim import corpora
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
from nltk.stem.porter import PorterStemmer
from bs4 import BeautifulSoup

['emails.csv']


In [74]:
import pandas as pd
import numpy as np
import wordcloud
import networkx as nx
import matplotlib.pyplot as plt
import os, email, sys, re

In [75]:
df_email = pd.read_csv('../input/emails.csv')
# df_email.head()

In [76]:
# df_email['message'].head()

In [77]:
def content_from_email(msg):
    parts = [];
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)


def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs

In [78]:
# Parse the emails into a list email objects
messages = list(map(email.message_from_string, df_email['message']))
df_email.drop('message', axis=1, inplace=True)

In [79]:
# df_email.head()

In [80]:
#  Get fields from parsed email objects
keys = messages[0].keys()
for key in keys:
    df_email[key] = [doc[key] for doc in messages]

In [81]:
# df_email.head()

In [82]:
# Parse content from emails
df_email['content'] = list(map(content_from_email, messages))
# df_email.head()

In [83]:
# Split multiple email addresses
df_email['From'] = df_email['From'].map(split_email_addresses)
df_email['To'] = df_email['To'].map(split_email_addresses)

# Extract the root of 'file' as 'user'
df_email['user'] = df_email['file'].map(lambda x:x.split('/')[0])
del messages

# df_email.head()

In [84]:
df_email.head()

,file,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user
0,allen-p/_sent_mail/1.,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",(phillip.allen@enron.com),(tim.belden@enron.com),,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast\n\n,allen-p
1,allen-p/_sent_mail/10.,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",(phillip.allen@enron.com),(john.lavorato@enron.com),Re:,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,allen-p
2,allen-p/_sent_mail/100.,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",(phillip.allen@enron.com),(leah.arsdall@enron.com),Re: test,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Leah Van Arsdall,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,test successful. way to go!!!,allen-p
3,allen-p/_sent_mail/1000.,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",(phillip.allen@enron.com),(randall.gay@enron.com),,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Randall L Gay,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Randy,\n\n Can you send me a schedule of the s...",allen-p
4,allen-p/_sent_mail/1001.,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",(phillip.allen@enron.com),(greg.piper@enron.com),Re: Hello,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Let's shoot for Tuesday at 11:45.,allen-p


In [85]:
# a = df_email.drop(['file', 'Mime-Version', 'Content-Type', 'Content-Transfer-Encoding'], inplace=True, axis=1)

In [86]:
# df_email.head()

In [ ]:
new_email_copy = df_email_email.copy()

In [95]:
# Setting up the index of the dataset
df_email.set_index('Message-ID', inplace = True)

In [96]:
# def clean(text):
#     stop = set(stopwords.words('english'))
#     stop.update(("to","cc","subject","http","from","sent","aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"))
#     exclude = set(string.punctuation) 
#     lemma = WordNetLemmatizer()
#     porter= PorterStemmer()
    
#     text=text.rstrip()
#     text = re.sub(r'[^a-zA-Z]', ' ', text)
#     stop_free = " ".join([i for i in text.lower().split() if((i not in stop) and (not i.isdigit()))])
#     punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
#     normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
#     #stem = " ".join(porter.stem(token) for token in normalized.split())
    
#     return normalized

def remove_stopwords(review_text):
#     lemma = WordNetLemmatizer()
#     porter= PorterStemmer()
    clean_text = BeautifulSoup(review_text) # Initialising Beautifulsoup
    get_clean_text = clean_text.get_text()  # Extract the text while removing the Mark-ups
    get_letters = re.sub("[^a-zA-Z]", " ", get_clean_text)  # Removing all numbers from the text
    stop = set(stopwords.words('english'))
    stop.update(("to","cc","subject","http","from","sent","aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"))
    clean_text = get_letters.lower()  # Convert all words to lower case
    clean_text = clean_text.split()   # Spliting all words to an individual words
    clean_text = [w for w in clean_text if not w in stopwords.words("english")]   # Removing stop words with the help of NLTK lib
    exclude = set(string.punctuation) 
    clean_text = [w for w in clean_text if not w in exclude]
    combines_clean_text = ' '.join(clean_text)   # Combining the words after removing stop-words
    return combines_clean_text

In [99]:
new_df = new_email_copy[['Message-ID','content']]

In [100]:
list(new_df.columns.values)

['Message-ID', 'content']

In [103]:
new_df['content'].head()

0                            Here is our forecast\n\n 
1    Traveling to have a business meeting takes the...
2                       test successful.  way to go!!!
3    Randy,\n\n Can you send me a schedule of the s...
4                  Let's shoot for Tuesday at 11:45.  
Name: content, dtype: object

In [105]:
total_review = new_df.shape[0]
total_review

517401

In [ ]:
all_clean_reviews = []
for i in range(0, total_review):
    clean_reviews = remove_stopwords(new_df['content'][i])
    print("Review Completed: %d out of %d" % (i+1, total_review))
    all_clean_reviews.append(clean_reviews)